# Convolutional Neural Networks
Convolutional neural networks are a form of neural network architecture that allow us to keep track of spatial relationship between individual input features. This is particularly useful for images, because to be able to make inference on images you would need to have information of the surrounding pixels on a particular image pixel to be able to identify what the pixel represents. This has made ConvNets the defacto architecture for convolutional neural networks. 

### Connected Layers to Convolutions
The journey from fully connected layers to convolutions and why they are important

In [2]:
import tensorflow as tf
from d2l import tensorflow as d2l
import numpy as np

In [5]:
a1 = np.array([[0, 1], [3, 4]])
a2 = np.array([[0, 1], [2, 3]])
mul_result = np.multiply(a1, a2)
np.sum(mul_result)

19

In [7]:
a1.shape

(2, 2)

In [6]:
def single_conv(t1, t2):
    mul_result = np.multiply(t1, t2)
    return np.sum(mul_result)

In [37]:
def full_conv(data, kernel):
    data_height, data_width = data.shape
    kernel_height, kernel_width = kernel.shape
    
    target = np.zeros(shape=(((data_height-kernel_height)+1), 
                             ((data_width-kernel_width)+1)))
    
    
    for i in range(0, (data_height-kernel_height)+1):
        for j in range(0, (data_width-kernel_width)+1):
            temp = input[i:kernel_height+i, j:kernel_width+j]
            result = single_conv(temp, kernel)
            target[i, j] = result 
    return target
            
data = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
kernel = np.array([[0, 1], [2, 3]])

In [38]:
full_conv(data, kernel)

array([[19., 25.],
       [37., 43.]])

In [39]:
def corr2d(X, K):
    # compute 2D cross-correlation
    h, w = K.shape
    Y = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j].assign(tf.reduce_sum(X[i:i+h, j:j+w] * K))
    return Y

In [40]:
X = tf.constant([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = tf.constant([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[19., 25.],
       [37., 43.]], dtype=float32)>

### Constructing a Convolutional Layer in tensorflow
We would be looking out how to implment a full convolutional operation layer using the functions we have implemented earlier

In [ ]:
class Conv2D(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
    
    def build(self, kernel_size):
        initializer = tf.random_uniform_initializer()
        self.weights = self.add_weight(name="w", 
                                       shape=kernel_size, 
                                       initializer=initializer)
        self.bias = self.add_weight(name="b", shape=(1,), 
                                    initializer=initializer)
        
    def call(self, inputs):
        return corr2d(inputs, self.weights) + self.bias 

### Object Edge detection in images
We will examine how convolutional neural networks can be used to detect teh objects in an image

In [41]:
X = tf.Variable(tf.ones((6, 8)))
X[:, 2:6].assign(tf.zeros(X[:, 2:6].shape))
X

<tf.Variable 'Variable:0' shape=(6, 8) dtype=float32, numpy=
array([[1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.]], dtype=float32)>

In [42]:
K = tf.constant([[1.0, -1.0]])

In [44]:
# perform cross correlation with simple finite edge detector
Y = corr2d(X, K)
Y

<tf.Variable 'Variable:0' shape=(6, 7) dtype=float32, numpy=
array([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.]], dtype=float32)>

In [45]:
corr2d(tf.transpose(X), K)

<tf.Variable 'Variable:0' shape=(8, 5) dtype=float32, numpy=
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)>

### Learning a Kernel
As we build larger kernels for deeper layers it would become more difficult to explicitly determine and program the functionality of the kerel. Thereforre we wil use iterative gradient algorithms to learn the parameters of a kernl

In [51]:
# Construct a two-dimensional convolutional layer with 1 output channel and a
# kernel of shape (1, 2). For the sake of simplicity, we ignore the bias here
conv2d = tf.keras.layers.Conv2D(1, (1, 2), use_bias=False)

# The two-dimensional convolutional layer uses four-dimensional input and
# output in the format of (example, height, width, channel), where the batch
# size (number of examples in the batch) and the number of channels are both 1
X = tf.reshape(X, (1, 6, 8, 1))
Y = tf.reshape(Y, (1, 6, 7, 1))
lr = 3e-2  # Learning rate

Y_hat = conv2d(X)
for i in range(10):
    with tf.GradientTape(watch_accessed_variables=False) as g:
        g.watch(conv2d.weights[0])
        Y_hat = conv2d(X)
        l = (abs(Y_hat - Y)) ** 2
        # Update the kernel
        update = tf.multiply(lr, g.gradient(l, conv2d.weights[0]))
        weights = conv2d.get_weights()
        weights[0] = conv2d.weights[0] - update
        conv2d.set_weights(weights)
        if (i + 1) % 2 == 0:
            print(f'epoch {i + 1}, loss {tf.reduce_sum(l):.3f}')

epoch 2, loss 19.738
epoch 4, loss 7.252
epoch 6, loss 2.831
epoch 8, loss 1.136
epoch 10, loss 0.461


In [60]:
conv2d = tf.keras.layers.Conv2D(1, (1, 2), use_bias=False)

X = tf.reshape(X, (1, 6, 8, 1))
Y = tf.reshape(Y, (1, 6, 7, 1))
lr = 3e-2

Y_hat = conv2d(X)

for i in range(10):
    with tf.GradientTape(watch_accessed_variables=False) as g:
        g.watch(conv2d.weights[0])
        Y_hat = conv2d(X)
#         # l = (abs(Y_hat - Y)) ** 2
#         update = tf.multiply(lr, g.gradient(l, conv2d.weights[0]))
#         weights = conv2d.get_weights()
#         weights[0] = conv2d.weights[0] - update
#         conv2d.set_weights(weights)
        
#         if (i + 1) %  2 == 0:
#             print(f'epoch {i + 1}, loss {tf.reduce_sum(l):.3f}')

# for i in range(10):
    # with tf.GradientTape(watch_accessed_variables=False) as g:
        # g.watch(conv2d.weights[0])
        # Y_hat = conv2d(X)
        l = (abs(Y_hat - Y)) ** 2
        # Update the kernel
        update = tf.multiply(lr, g.gradient(l, conv2d.weights[0]))
        weights = conv2d.get_weights()
        weights[0] = conv2d.weights[0] - update
        conv2d.set_weights(weights)
        if (i + 1) % 2 == 0:
            print(f'epoch {i + 1}, loss {tf.reduce_sum(l):.3f}')



epoch 2, loss 0.768
epoch 4, loss 0.287
epoch 6, loss 0.113
epoch 8, loss 0.046
epoch 10, loss 0.019
